<a href="https://colab.research.google.com/github/WilliamPoe/CSCI-290/blob/main/notebooks/asm_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import math